In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster-2"
compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes=4)
compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0,2),
        "--max_iter": choice(25,50,75,100,125,150,175,200)
    })### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            arguments=['--C', 1, '--max_iter', 100],
                            compute_target=compute_target,
                            environment=sklearn_env) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=10,
                             max_concurrent_runs=4)### YOUR CODE HERE ###

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyper_drive= exp.submit(config=hyperdrive_config)
RunDetails(hyper_drive).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
hyper_drive.wait_for_completion(show_output=True)

RunId: HD_d0c321cc-79c7-4b4d-ba6a-a08e2f35cd6f
Web View: https://ml.azure.com/runs/HD_d0c321cc-79c7-4b4d-ba6a-a08e2f35cd6f?wsid=/subscriptions/7c03dd83-6b95-43b1-9f53-23dfd07e8803/resourcegroups/azp-078-rg/workspaces/ai-tci-ml&tid=eb70b763-b6d7-4486-8555-8831709a784e

Streaming azureml-logs/hyperdrive.txt

[2024-02-07T04:54:46.060192][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-02-07T04:54:46.6499251Z][SCHEDULER][INFO]Scheduling job, id='HD_d0c321cc-79c7-4b4d-ba6a-a08e2f35cd6f_0' 
[2024-02-07T04:54:46.7815527Z][SCHEDULER][INFO]Scheduling job, id='HD_d0c321cc-79c7-4b4d-ba6a-a08e2f35cd6f_1' 
[2024-02-07T04:54:47.0381140Z][SCHEDULER][INFO]Scheduling job, id='HD_d0c321cc-79c7-4b4d-ba6a-a08e2f35cd6f_3' 
[2024-02-07T04:54:46.9178549Z][SCHEDULER][INFO]Scheduling job, id='HD_d0c321cc-79c7-4b4d-ba6a-a08e2f35cd6f_2' 
[2024-02-07T04:54:46.984418][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2024-02-07T04:54:

{'runId': 'HD_d0c321cc-79c7-4b4d-ba6a-a08e2f35cd6f',
 'target': 'keshav-ds11',
 'status': 'Completed',
 'startTimeUtc': '2024-02-07T04:54:45.232141Z',
 'endTimeUtc': '2024-02-07T05:00:20.833254Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0d62cb01-fdf2-47fd-8ea9-074135f5ef1d',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_d0c321cc-79c7-4b4d-ba6a-a08e2f35cd6f_0',
  'score': '0.9116240573845872',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_d0c321cc-79c7-4b4d-ba6a-a08e2f35cd6f_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValu

In [27]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyper_drive.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:',parameter_values[5])
print('\n max_iter:',parameter_values[7])
# print('\n batch size:',parameter_values[7])

Best Run Id:  HD_84bd0dd8-6c2e-42c8-a5e8-d15fb166e8f0_7

 Accuracy: 0.9116240573845871

 C: 0.544367502254389

 max_iter: 50


In [28]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv",header=True, validate=True)

In [30]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###)
x['target'] = y

In [31]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=x,
    label_column_name= 'target',
    n_cross_validations=2)

In [32]:
# Submit your automl run

### YOUR CODE HERE ###
autoML_exp = Experiment(workspace=ws, name="udacity-automl")
autoML_run = autoML_exp.start_logging()
run_auto = autoML_exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-automl,AutoML_2c9f54ca-8fc2-4d48-96fc-698e4ab53591,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2024/02/06 13:43:21 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024-02-06:14:05:15,894 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-02-06:14:05:17,106 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-02-06:14:05:17,107 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-02-06:14:05:18,362 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-02-06:

In [34]:
run_auto.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-automl,AutoML_2c9f54ca-8fc2-4d48-96fc-698e4ab53591,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_2c9f54ca-8fc2-4d48-96fc-698e4ab53591',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2024-02-06T13:34:55.065573Z',
 'endTimeUtc': '2024-02-06T14:05:31.932195Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-automl","subscription_id":"7c03dd83-6b95-43b1-9f53-23dfd07e8803","resource_group":"azp-078-rg","workspace_name":"ai-tci-ml","region":"westeurope","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"

In [35]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_auto_run = run_auto.get_best_child()
best_auto_run_metrics = best_auto_run.get_metrics()

In [38]:
auto_model = best_auto_run.register_model(model_name='autML_model', model_path='/', 
                                properties={'Accuracy': best_auto_run_metrics['accuracy']})